# `Ragas API Client`

> Python client to api.ragas.io

In [1]:
#| default_exp backends.ragas_api_client

In [1]:
RAGAS_APP_TOKEN = "apt.47bd-c55e4a45b27c-02f8-8446-1441f09b-651a8"
RAGAS_API_ENDPOINT = "https://api.dev.app.ragas.io"

In [6]:
#| export
import httpx
import asyncio
import typing as t
from pydantic import BaseModel, Field
from enum import StrEnum
import uuid
from fastcore.utils import patch

In [5]:
#| export
class RagasRelay:
    """Client for the Ragas Relay API."""

    def __init__(self, base_url: str, app_token: t.Optional[str] = None):
        """Initialize the Ragas API client.
        
        Args:
            base_url: Base URL for the API (e.g., "http://localhost:8087")
            app_token: API token for authentication
        """
        if not app_token:
            raise ValueError("app_token must be provided")

        self.base_url = f"{base_url.rstrip('/')}/api/v1"
        self.app_token = app_token

    async def _request(
        self,
        method: str,
        endpoint: str,
        params: t.Optional[t.Dict] = None,
        json_data: t.Optional[t.Dict] = None,
    ) -> t.Dict:
        """Make a request to the API.
        
        Args:
            method: HTTP method (GET, POST, PATCH, DELETE)
            endpoint: API endpoint path
            params: Query parameters
            json_data: JSON request body
            
        Returns:
            The response data from the API
        """
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        headers = {"X-App-Token": self.app_token}

        async with httpx.AsyncClient() as client:
            response = await client.request(
                method=method, url=url, params=params, json=json_data, headers=headers
            )

            data = response.json()

            if response.status_code >= 400 or data.get("status") == "error":
                error_msg = data.get("message", "Unknown error")
                raise Exception(f"API Error ({response.status_code}): {error_msg}")

            return data.get("data")

    #---- Resource Handlers ----
    async def _create_resource(self, path, data):
        """Generic resource creation."""
        return await self._request("POST", path, json_data=data)
        
    async def _list_resources(self, path, **params):
        """Generic resource listing."""
        return await self._request("GET", path, params=params)
        
    async def _get_resource(self, path):
        """Generic resource retrieval."""
        return await self._request("GET", path)
        
    async def _update_resource(self, path, data):
        """Generic resource update."""
        return await self._request("PATCH", path, json_data=data)
        
    async def _delete_resource(self, path):
        """Generic resource deletion."""
        return await self._request("DELETE", path)

In [7]:
#| export
#---- Projects ----
@patch
async def list_projects(
    self: RagasRelay,
    ids: t.Optional[t.List[str]] = None,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List projects."""
    params = {"limit": limit, "offset": offset}

    if ids:
        params["ids"] = ",".join(ids)

    if order_by:
        params["order_by"] = order_by

    if sort_dir:
        params["sort_dir"] = sort_dir

    return await self._list_resources("projects", **params)

@patch
async def get_project(self: RagasRelay, project_id: str) -> t.Dict:
    """Get a specific project by ID."""
    return await self._get_resource(f"projects/{project_id}")

@patch
async def create_project(
    self: RagasRelay, title: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new project."""
    data = {"title": title}
    if description:
        data["description"] = description
    return await self._create_resource("projects", data)

@patch
async def update_project(
    self: RagasRelay,
    project_id: str,
    title: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing project."""
    data = {}
    if title:
        data["title"] = title
    if description:
        data["description"] = description
    return await self._update_resource(f"projects/{project_id}", data)

@patch
async def delete_project(self: RagasRelay, project_id: str) -> None:
    """Delete a project."""
    await self._delete_resource(f"projects/{project_id}")


In [8]:
# Initialize client with your authentication token
client = RagasRelay(base_url=RAGAS_API_ENDPOINT, app_token=RAGAS_APP_TOKEN)

# List projects
try:
    projects = await client.list_projects(limit=10)
    print(f"Found {len(projects)} projects:")
    for project in projects:
        print(f"- {project['title']} (ID: {project['id']})")
except Exception as e:
    print(f"Error: {e}")

Found 2 projects:
Error: string indices must be integers, not 'str'


### Projects

In [9]:
await client.create_project("test project", "test description")

{'id': '59cf2483-d2c7-4306-af87-bfac2813f27b',
 'title': 'test project',
 'description': 'test description',
 'created_at': '2025-04-09T05:57:57.991728+00:00',
 'updated_at': '2025-04-09T05:57:57.991728+00:00'}

In [10]:
await client.list_projects()

{'items': [{'id': '59cf2483-d2c7-4306-af87-bfac2813f27b',
   'title': 'test project',
   'description': 'test description',
   'created_at': '2025-04-09T05:57:57.991728+00:00',
   'updated_at': '2025-04-09T05:57:57.991728+00:00'},
  {'id': 'c026b63c-d618-42c0-81c3-d7824c976eb1',
   'title': 'test project',
   'description': 'test description',
   'created_at': '2025-04-08T22:46:04.045516+00:00',
   'updated_at': '2025-04-08T22:46:04.045516+00:00'},
  {'id': '3dd738de-49f7-494c-aa0a-f6531d3b603a',
   'title': 'RagasTest',
   'description': '',
   'created_at': '2025-04-08T17:45:32.759553+00:00',
   'updated_at': '2025-04-08T17:45:32.759553+00:00'},
  {'id': '2f45d026-1b13-4851-a36d-c7680edb6380',
   'title': 'test project',
   'description': 'test description',
   'created_at': '2025-04-08T14:38:14.61165+00:00',
   'updated_at': '2025-04-08T14:38:14.61165+00:00'},
  {'id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6',
   'title': 'test project',
   'description': 'test description',
   'creat

In [11]:
TEST_PROJECT_ID = "e1b3f1e4-d344-48f4-a178-84e7e32e6ab6"
project = await client.get_project(TEST_PROJECT_ID)

### Datasets

In [12]:
#| export

#---- Datasets ----
@patch
async def list_datasets(
    self: RagasRelay,
    project_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List datasets in a project."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(f"projects/{project_id}/datasets", **params)

@patch
async def get_dataset(self: RagasRelay, project_id: str, dataset_id: str) -> t.Dict:
    """Get a specific dataset."""
    return await self._get_resource(f"projects/{project_id}/datasets/{dataset_id}")

@patch
async def create_dataset(
    self: RagasRelay, project_id: str, name: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new dataset in a project."""
    data = {"name": name}
    if description:
        data["description"] = description
    return await self._create_resource(f"projects/{project_id}/datasets", data)

@patch
async def update_dataset(
    self: RagasRelay,
    project_id: str,
    dataset_id: str,
    name: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing dataset."""
    data = {}
    if name:
        data["name"] = name
    if description:
        data["description"] = description
    return await self._update_resource(f"projects/{project_id}/datasets/{dataset_id}", data)

@patch
async def delete_dataset(self: RagasRelay, project_id: str, dataset_id: str) -> None:
    """Delete a dataset."""
    await self._delete_resource(f"projects/{project_id}/datasets/{dataset_id}")

In [13]:
# check project ID
projects = await client.list_projects()
projects["items"][0]["id"], TEST_PROJECT_ID

('59cf2483-d2c7-4306-af87-bfac2813f27b',
 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6')

In [14]:
# Create a new dataset
new_dataset = await client.create_dataset(
    projects["items"][0]["id"], "New Dataset", "This is a new dataset"
)
print(f"New dataset created: {new_dataset}")

New dataset created: {'id': '656e632c-6f0b-4046-929e-05b98fb14eaa', 'name': 'New Dataset', 'description': 'This is a new dataset', 'updated_at': '2025-04-09T05:59:04.352084+00:00', 'created_at': '2025-04-09T05:59:04.352084+00:00', 'version_counter': 0, 'project_id': '59cf2483-d2c7-4306-af87-bfac2813f27b'}


In [15]:
# List datasets in the project
datasets = await client.list_datasets(projects["items"][0]["id"])
print(f"Found {len(datasets)} datasets")

Found 2 datasets


In [16]:
updated_dataset = await client.update_dataset(
    projects["items"][0]["id"],
    datasets["items"][0]["id"],
    "Updated Dataset",
    "This is an updated dataset",
)
print(f"Updated dataset: {updated_dataset}")

Updated dataset: {'id': '656e632c-6f0b-4046-929e-05b98fb14eaa', 'name': 'Updated Dataset', 'description': 'This is an updated dataset', 'created_at': '2025-04-09T05:59:04.352084+00:00', 'updated_at': '2025-04-09T05:59:10.388986+00:00', 'version_counter': 0, 'project_id': '59cf2483-d2c7-4306-af87-bfac2813f27b'}


In [17]:
# Delete the dataset
await client.delete_dataset(projects["items"][0]["id"], datasets["items"][0]["id"])
print("Dataset deleted")

Dataset deleted


### Experiments

In [18]:
 #| export
#---- Experiments ----
@patch
async def list_experiments(
    self: RagasRelay,
    project_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List experiments in a project."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(f"projects/{project_id}/experiments", **params)

@patch
async def get_experiment(self: RagasRelay, project_id: str, experiment_id: str) -> t.Dict:
    """Get a specific experiment."""
    return await self._get_resource(f"projects/{project_id}/experiments/{experiment_id}")

@patch
async def create_experiment(
    self: RagasRelay, project_id: str, name: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new experiment in a project."""
    data = {"name": name}
    if description:
        data["description"] = description
    return await self._create_resource(f"projects/{project_id}/experiments", data)

@patch
async def update_experiment(
    self: RagasRelay,
    project_id: str,
    experiment_id: str,
    name: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing experiment."""
    data = {}
    if name:
        data["name"] = name
    if description:
        data["description"] = description
    return await self._update_resource(f"projects/{project_id}/experiments/{experiment_id}", data)

@patch
async def delete_experiment(self: RagasRelay, project_id: str, experiment_id: str) -> None:
    """Delete an experiment."""
    await self._delete_resource(f"projects/{project_id}/experiments/{experiment_id}")


In [19]:
# create a new experiment
new_experiment = await client.create_experiment(
    projects["items"][0]["id"], "New Experiment", "This is a new experiment"
)
print(f"New experiment created: {new_experiment}")
# list experiments
experiments = await client.list_experiments(projects["items"][0]["id"])
print(f"Found {len(experiments)} experiments")
# get a specific experiment
experiment = await client.get_experiment(
    projects["items"][0]["id"], experiments["items"][0]["id"]
)
print(f"Experiment: {experiment}")
# update an experiment
updated_experiment = await client.update_experiment(
    projects["items"][0]["id"],
    experiments["items"][0]["id"],
    "Updated Experiment",
    "This is an updated experiment",
)
print(f"Updated experiment: {updated_experiment}")
# delete an experiment
await client.delete_experiment(projects["items"][0]["id"], experiments["items"][0]["id"])
print("Experiment deleted")

New experiment created: {'id': '783c34ec-9d92-4bf9-9a45-d64746e58ba9', 'name': 'New Experiment', 'description': 'This is a new experiment', 'updated_at': '2025-04-09T06:00:01.80579+00:00', 'created_at': '2025-04-09T06:00:01.80579+00:00', 'version_counter': 0, 'project_id': '59cf2483-d2c7-4306-af87-bfac2813f27b'}
Found 2 experiments
Experiment: {'id': '783c34ec-9d92-4bf9-9a45-d64746e58ba9', 'name': 'New Experiment', 'description': 'This is a new experiment', 'created_at': '2025-04-09T06:00:01.80579+00:00', 'updated_at': '2025-04-09T06:00:01.80579+00:00', 'version_counter': 0, 'project_id': '59cf2483-d2c7-4306-af87-bfac2813f27b'}
Updated experiment: {'id': '783c34ec-9d92-4bf9-9a45-d64746e58ba9', 'name': 'Updated Experiment', 'description': 'This is an updated experiment', 'created_at': '2025-04-09T06:00:01.80579+00:00', 'updated_at': '2025-04-09T06:00:04.394175+00:00', 'version_counter': 0, 'project_id': '59cf2483-d2c7-4306-af87-bfac2813f27b'}
Experiment deleted


In [20]:
await client.list_experiments(TEST_PROJECT_ID)

{'items': [{'id': '78fd6c58-7edf-4239-93d1-4f49185d8e49',
   'name': 'New Experiment',
   'description': 'This is a new experiment',
   'created_at': '2025-03-30T06:31:31.689269+00:00',
   'updated_at': '2025-03-30T06:31:31.689269+00:00',
   'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 1,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

### Columns (for datasets)

The API supports the following column types:

- `number`: Numeric values
- `longText`: Text content
- `select`: Single selection from predefined options
- `date`: Date values
- `multiSelect`: Multiple selections from predefined options
- `checkbox`: Boolean values
- `custom`: Custom column types with specific behavior

Each column type has specific settings that can be configured through the `settings` object.

In [80]:
#| export
class ColumnType(StrEnum):
    NUMBER = "number"
    TEXT = "text"
    LONG_TEXT = "longText"
    SELECT = "select"
    DATE = "date"
    MULTI_SELECT = "multiSelect"
    CHECKBOX = "checkbox"
    CUSTOM = "custom"

In [24]:
#| export

#---- Dataset Columns ----
@patch
async def list_dataset_columns(
    self: RagasRelay,
    project_id: str,
    dataset_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List columns in a dataset."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/datasets/{dataset_id}/columns", **params
    )

@patch
async def get_dataset_column(
    self: RagasRelay, project_id: str, dataset_id: str, column_id: str
) -> t.Dict:
    """Get a specific column in a dataset."""
    return await self._get_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}"
    )

@patch
async def create_dataset_column(
    self: RagasRelay,
    project_id: str,
    dataset_id: str,
    id: str,
    name: str,
    type: str,
    col_order: t.Optional[int] = None,
    settings: t.Optional[t.Dict] = None,
) -> t.Dict:
    """Create a new column in a dataset."""
    data = {"id": id, "name": name, "type": type}
    if col_order is not None:
        data["col_order"] = col_order
    if settings:
        data["settings"] = settings
    return await self._create_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns", data
    )

@patch
async def update_dataset_column(
    self: RagasRelay, project_id: str, dataset_id: str, column_id: str, **column_data
) -> t.Dict:
    """Update an existing column in a dataset."""
    return await self._update_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}",
        column_data,
    )

@patch
async def delete_dataset_column(
    self: RagasRelay, project_id: str, dataset_id: str, column_id: str
) -> None:
    """Delete a column from a dataset."""
    await self._delete_resource(
        f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}"
    )

In [25]:
datasets = await client.create_dataset(
    projects["items"][0]["id"],
    "New Dataset for testing columns",
    "This is a new dataset for testing columns",
)
datasets

{'id': '9f0f1ac6-beb6-4bda-95d4-9d2a7dc4b837',
 'name': 'New Dataset for testing columns',
 'description': 'This is a new dataset for testing columns',
 'updated_at': '2025-04-09T06:01:40.671173+00:00',
 'created_at': '2025-04-09T06:01:40.671173+00:00',
 'version_counter': 0,
 'project_id': '59cf2483-d2c7-4306-af87-bfac2813f27b'}

In [26]:
# add a new column to the dataset
new_column = await client.create_dataset_column(
    project_id=projects["items"][0]["id"],
    dataset_id=datasets["id"],
    id="new_column_3",
    name="New Column 3",
    type=ColumnType.TEXT.value,
    settings={
        "max_length": 255,
        "is_required": True,
    },
)
new_column

{'id': 'new_column_3',
 'name': 'New Column 3',
 'type': 'longText',
 'settings': {'id': 'new_column_3',
  'name': 'New Column 3',
  'type': 'longText',
  'max_length': 255,
  'is_required': True},
 'created_at': '2025-04-09T06:01:41.978634+00:00',
 'updated_at': '2025-04-09T06:01:41.978634+00:00',
 'datatable_id': '9f0f1ac6-beb6-4bda-95d4-9d2a7dc4b837'}

In [27]:
await client.list_dataset_columns(projects["items"][0]["id"], datasets["id"])

{'items': [{'id': 'new_column_3',
   'name': 'New Column 3',
   'type': 'longText',
   'settings': {'id': 'new_column_3',
    'name': 'New Column 3',
    'type': 'longText',
    'max_length': 255,
    'is_required': True},
   'created_at': '2025-04-09T06:01:41.978634+00:00',
   'updated_at': '2025-04-09T06:01:41.978634+00:00',
   'datatable_id': '9f0f1ac6-beb6-4bda-95d4-9d2a7dc4b837'}],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 1,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

In [28]:
col3 = await client.get_dataset_column(
    projects["items"][0]["id"], datasets["id"], "new_column_3"
)
col3

{'id': 'new_column_3',
 'name': 'New Column 3',
 'type': 'longText',
 'settings': {'id': 'new_column_3',
  'name': 'New Column 3',
  'type': 'longText',
  'max_length': 255,
  'is_required': True},
 'created_at': '2025-04-09T06:01:41.978634+00:00',
 'updated_at': '2025-04-09T06:01:41.978634+00:00',
 'datatable_id': '9f0f1ac6-beb6-4bda-95d4-9d2a7dc4b837'}

In [29]:
await client.update_dataset_column(
    projects["items"][0]["id"],
    datasets["id"],
    "new_column_3",
    name="New Column 3 Updated",
    type=ColumnType.NUMBER.value,
)

{'id': 'new_column_3',
 'name': 'New Column 3 Updated',
 'type': 'number',
 'settings': {'id': 'new_column_3',
  'name': 'New Column 3',
  'type': 'longText',
  'max_length': 255,
  'is_required': True},
 'created_at': '2025-04-09T06:01:41.978634+00:00',
 'updated_at': '2025-04-09T06:01:46.946099+00:00',
 'datatable_id': '9f0f1ac6-beb6-4bda-95d4-9d2a7dc4b837'}

In [30]:
await client.delete_dataset_column(
    projects["items"][0]["id"], datasets["id"], "new_column_3"
)

### Rows (for datasets)

In [35]:
#---- Dataset Rows ----
@patch
async def list_dataset_rows(
    self: RagasRelay,
    project_id: str,
    dataset_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List rows in a dataset."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/datasets/{dataset_id}/rows", **params
    )

@patch
async def get_dataset_row(
    self: RagasRelay, project_id: str, dataset_id: str, row_id: str
) -> t.Dict:
    """Get a specific row in a dataset."""
    return await self._get_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}"
    )

@patch
async def create_dataset_row(
    self: RagasRelay, project_id: str, dataset_id: str, id: str, data: t.Dict
) -> t.Dict:
    """Create a new row in a dataset."""
    row_data = {"id": id, "data": data}
    return await self._create_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows", row_data
    )

@patch
async def update_dataset_row(
    self: RagasRelay, project_id: str, dataset_id: str, row_id: str, data: t.Dict
) -> t.Dict:
    """Update an existing row in a dataset."""
    row_data = {"data": data}
    return await self._update_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}",
        row_data,
    )

@patch
async def delete_dataset_row(
    self: RagasRelay, project_id: str, dataset_id: str, row_id: str
) -> None:
    """Delete a row from a dataset."""
    await self._delete_resource(
        f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}"
    )


In [36]:
await client.create_dataset_row(
    projects["items"][0]["id"],
    datasets["id"],
    "1",
    {"name": "New Row 1", "age": 30},
)


Exception: API Error (500): Failed to create row

In [37]:
await client.list_dataset_rows(projects["items"][0]["id"], datasets["id"])

{'items': [],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 0,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

In [40]:
#| export
import uuid
import string

In [41]:
#| export
def create_nano_id(size=12):
    # Define characters to use (alphanumeric)
    alphabet = string.ascii_letters + string.digits
    
    # Generate UUID and convert to int
    uuid_int = uuid.uuid4().int
    
    # Convert to base62
    result = ""
    while uuid_int:
        uuid_int, remainder = divmod(uuid_int, len(alphabet))
        result = alphabet[remainder] + result
    
    # Pad if necessary and return desired length
    return result[:size]


In [42]:
# Usage
nano_id = create_nano_id()  # e.g., "8dK9cNw3mP5x"
nano_id

'eAscOAbBMSv2'

### Get a Dataset Visualized - Created From UI
Lets Create a new dataset and add columns and rows via the endpoint to see how it behaves

In [52]:
# generate a dataset
dataset = await client.create_dataset(
    project_id=TEST_PROJECT_ID,
    name="Dataset Visualized from UI",
    description="This is a dataset created from the UI",
)

# show url
WEB_ENDPOINT = "https://dev.app.ragas.io"
url = f"{WEB_ENDPOINT}/dashboard/projects/{TEST_PROJECT_ID}/datasets/{dataset['id']}"
url

'https://dev.app.ragas.io/dashboard/projects/e1b3f1e4-d344-48f4-a178-84e7e32e6ab6/datasets/cc239f73-6546-4d5c-84e1-8c2ed8072edc'

In [49]:
# list columns
columns = await client.list_dataset_columns(TEST_PROJECT_ID, dataset["id"])
# list rows
rows = await client.list_dataset_rows(TEST_PROJECT_ID, dataset["id"])

In [50]:
columns


{'items': [],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 0,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

In [51]:
rows

{'items': [],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 0,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

### Create a Dataset from data

we want to be able to use the API with python data like this `t.List[t.Dict]`.
```py
# how we want the data to look
data = [
    {
        "id": "1",
        "query": "What is the capital of France?",
        "persona": "John",
        "ground_truth": "Paris",
    },
    {
        "id": "2",
        "query": "What is the capital of Germany?",
        "persona": "Jane",
        "ground_truth": "Berlin",
    },
    {
        "id": "3",
        "query": "What is the capital of Italy?",
        "persona": "John",
        "ground_truth": "Rome",
    },
]
```

In [81]:
# print out column types
print([col.value for col in ColumnType])

['number', 'text', 'longText', 'select', 'date', 'multiSelect', 'checkbox', 'custom']


In [44]:
# it should be able to handle simple python dicts
data = [
    {
        "id": "1",
        "query": "What is the capital of France?",
        "persona": "John",
        "ground_truth": "Paris",
    },
    {
        "id": "2",
        "query": "What is the capital of Germany?",
        "persona": "Jane",
        "ground_truth": "Berlin",
    },
]

There can be 2 ways to pass in data

1. Data can come as either as simple dicts

```py
data = [
    {"column_1": "value", "column_2": "value"}
]
```

2. or if you want to give more settings

```py
data = [
    {
        "column_1": {"data": "value", "type": ColumnType.text},
        "column_2": {"data": "value", "type": ColumnType.number},
    }
]
```

3. after that you will have to pass a list `Column` and `Row` to add it.

In [82]:
# test data
test_data_columns = [
    {"name": "id", "type": ColumnType.NUMBER.value},
    {"name": "query", "type": ColumnType.TEXT.value},
    {"name": "persona", "type": ColumnType.TEXT.value},
    {"name": "ground_truth", "type": ColumnType.TEXT.value},
]

test_data_rows = [{
    "id": "1",
    "query": "What is the capital of France?",
    "persona": "John",
    "ground_truth": "Paris",
}, {
    "id": "2",
    "query": "What is the capital of Germany?",
    "persona": "Jane",
    "ground_truth": "Berlin",
}, {
    "id": "3",
    "query": "What is the capital of Italy?",
    "persona": "John",
    "ground_truth": "Rome",
}]



In [83]:
#| export
# Default settings for columns
DEFAULT_SETTINGS = {
    "is_required": False,
    "max_length": 1000
}

# Model definitions
class Column(BaseModel):
    id: str = Field(default_factory=create_nano_id)
    name: str = Field(...)
    type: str = Field(...)
    settings: t.Dict = Field(default_factory=lambda: DEFAULT_SETTINGS.copy())
    col_order: t.Optional[int] = Field(default=None)

class RowCell(BaseModel):
    data: t.Any = Field(...)
    column_id: str = Field(...)

class Row(BaseModel):
    id: str = Field(default_factory=create_nano_id)
    data: t.List[RowCell] = Field(...)

In [84]:
#| export
#---- Resource With Data Helper Methods ----
@patch
async def _create_with_data(
    self: RagasRelay,
    resource_type: str,
    project_id: str,
    name: str, 
    description: str,
    columns: t.List[Column],
    rows: t.List[Row],
    batch_size: int = 50
) -> t.Dict:
    """Generic method to create a resource with columns and rows.
    
    Args:
        resource_type: Type of resource ("dataset" or "experiment")
        project_id: Project ID
        name: Resource name
        description: Resource description
        columns: List of column definitions
        rows: List of row data
        batch_size: Number of operations to perform concurrently
        
    Returns:
        The created resource
    """
    # Select appropriate methods based on resource type
    if resource_type == "dataset":
        create_fn = self.create_dataset
        create_col_fn = self.create_dataset_column
        create_row_fn = self.create_dataset_row
        delete_fn = self.delete_dataset
        id_key = "dataset_id"
    elif resource_type == "experiment":
        create_fn = self.create_experiment
        create_col_fn = self.create_experiment_column
        create_row_fn = self.create_experiment_row
        delete_fn = self.delete_experiment
        id_key = "experiment_id"
    else:
        raise ValueError(f"Unsupported resource type: {resource_type}")
        
    try:
        # Create the resource
        resource = await create_fn(project_id, name, description)
        
        # Process columns in batches
        for i in range(0, len(columns), batch_size):
            batch = columns[i:i+batch_size]
            col_tasks = []
            
            for col in batch:
                params = {
                    "project_id": project_id,
                    id_key: resource["id"], # dataset_id here
                    "id": col.id,
                    "name": col.name,
                    "type": col.type,
                    "settings": col.settings
                }
                if col.col_order is not None:
                    params["col_order"] = col.col_order
                
                col_tasks.append(create_col_fn(**params))
            
            await asyncio.gather(*col_tasks)
            
        # Process rows in batches
        for i in range(0, len(rows), batch_size):
            batch = rows[i:i+batch_size]
            row_tasks = []
            
            for row in batch:
                row_data = {cell.column_id: cell.data for cell in row.data}
                row_tasks.append(
                    create_row_fn(
                        project_id=project_id,
                        **{id_key: resource["id"]},
                        id=row.id,
                        data=row_data
                    )
                )
            
            await asyncio.gather(*row_tasks)
            
        return resource
        
    except Exception as e:
        # Clean up on error
        if 'resource' in locals():
            try:
                await delete_fn(project_id, resource["id"])
            except:
                pass  # Ignore cleanup errors
        raise e

@patch
async def create_dataset_with_data(
    self: RagasRelay,
    project_id: str,
    name: str,
    description: str,
    columns: t.List[Column],
    rows: t.List[Row],
    batch_size: int = 50
) -> t.Dict:
    """Create a dataset with columns and rows.
    
    This method creates a dataset and populates it with columns and rows in an
    optimized way using concurrent requests.
    
    Args:
        project_id: Project ID
        name: Dataset name
        description: Dataset description
        columns: List of column definitions
        rows: List of row data
        batch_size: Number of operations to perform concurrently
        
    Returns:
        The created dataset
    """
    return await self._create_with_data(
        "dataset", project_id, name, description, columns, rows, batch_size
    )

Now lets test this.

In [86]:
# First convert the raw data into proper Column and Row objects
from pydantic import BaseModel, Field
import typing as t

# Create Column objects
column_objects = []
for col in test_data_columns:
    column_objects.append(Column(
        name=col["name"],
        type=col["type"]
        # id and settings will be auto-generated
    ))

# Create a mapping of column names to their IDs for creating rows
column_map = {col.name: col.id for col in column_objects}

# Create Row objects
row_objects = []
for row in test_data_rows:
    cells = []
    for key, value in row.items():
        if key in column_map:  # Skip any extra fields not in columns
            cells.append(RowCell(
                data=value,
                column_id=column_map[key]
            ))
    row_objects.append(Row(data=cells))

# Now we can create the dataset
dataset = await client.create_dataset_with_data(
    project_id=TEST_PROJECT_ID,
    name="Capitals Dataset",
    description="A dataset about capital cities",
    columns=column_objects,
    rows=row_objects
)

print(f"Created dataset with ID: {dataset['id']}")

# Verify the data
columns = await client.list_dataset_columns(TEST_PROJECT_ID, dataset["id"])
print(f"Created {len(columns['items'])} columns")

rows = await client.list_dataset_rows(TEST_PROJECT_ID, dataset["id"])
print(f"Created {len(rows['items'])} rows")

Created dataset with ID: 5e7912f4-6a65-4d0c-bf79-0fab9ddda40c
Created 4 columns
Created 3 rows


In [87]:
# get dataset url
url = f"{WEB_ENDPOINT}/dashboard/projects/{TEST_PROJECT_ID}/datasets/{dataset['id']}"
url

'https://dev.app.ragas.io/dashboard/projects/e1b3f1e4-d344-48f4-a178-84e7e32e6ab6/datasets/5e7912f4-6a65-4d0c-bf79-0fab9ddda40c'

In [88]:
# cleanup
await client.delete_dataset(TEST_PROJECT_ID, dataset["id"])

### The same but for Experiments

In [89]:
#| export
#---- Experiment Columns ----
@patch
async def list_experiment_columns(
    self: RagasRelay,
    project_id: str,
    experiment_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List columns in an experiment."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/experiments/{experiment_id}/columns", **params
    )

@patch
async def get_experiment_column(
    self: RagasRelay, project_id: str, experiment_id: str, column_id: str
) -> t.Dict:
    """Get a specific column in an experiment."""
    return await self._get_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}"
    )

@patch
async def create_experiment_column(
    self: RagasRelay,
    project_id: str,
    experiment_id: str,
    id: str,
    name: str,
    type: str,
    col_order: t.Optional[int] = None,
    settings: t.Optional[t.Dict] = None,
) -> t.Dict:
    """Create a new column in an experiment."""
    data = {"id": id, "name": name, "type": type}
    if col_order is not None:
        data["col_order"] = col_order
    if settings:
        data["settings"] = settings
    return await self._create_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns", data
    )

@patch
async def update_experiment_column(
    self: RagasRelay, project_id: str, experiment_id: str, column_id: str, **column_data
) -> t.Dict:
    """Update an existing column in an experiment."""
    return await self._update_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}",
        column_data,
    )

@patch
async def delete_experiment_column(
    self: RagasRelay, project_id: str, experiment_id: str, column_id: str
) -> None:
    """Delete a column from an experiment."""
    await self._delete_resource(
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}"
    )

#---- Experiment Rows ----
@patch
async def list_experiment_rows(
    self: RagasRelay,
    project_id: str,
    experiment_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List rows in an experiment."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._list_resources(
        f"projects/{project_id}/experiments/{experiment_id}/rows", **params
    )

@patch
async def get_experiment_row(
    self: RagasRelay, project_id: str, experiment_id: str, row_id: str
) -> t.Dict:
    """Get a specific row in an experiment."""
    return await self._get_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}"
    )

@patch
async def create_experiment_row(
    self: RagasRelay, project_id: str, experiment_id: str, id: str, data: t.Dict
) -> t.Dict:
    """Create a new row in an experiment."""
    row_data = {"id": id, "data": data}
    return await self._create_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows", row_data
    )

@patch
async def update_experiment_row(
    self: RagasRelay, project_id: str, experiment_id: str, row_id: str, data: t.Dict
) -> t.Dict:
    """Update an existing row in an experiment."""
    row_data = {"data": data}
    return await self._update_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}",
        row_data,
    )

@patch
async def delete_experiment_row(
    self: RagasRelay, project_id: str, experiment_id: str, row_id: str
) -> None:
    """Delete a row from an experiment."""
    await self._delete_resource(
        f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}"
    )

In [90]:
await client.create_experiment(TEST_PROJECT_ID, "New Experiment", "This is a new experiment")

{'id': '7c695b58-7fc3-464c-a18b-a96e35f9684d',
 'name': 'New Experiment',
 'description': 'This is a new experiment',
 'updated_at': '2025-04-09T17:03:44.340782+00:00',
 'created_at': '2025-04-09T17:03:44.340782+00:00',
 'version_counter': 0,
 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}

In [91]:
experiments = await client.list_experiments(TEST_PROJECT_ID)
EXPERIMENT_ID = experiments["items"][0]["id"]
EXPERIMENT_ID

'78fd6c58-7edf-4239-93d1-4f49185d8e49'

In [93]:
#| export
@patch
async def create_experiment_with_data(
    self: RagasRelay,
    project_id: str,
    name: str,
    description: str,
    columns: t.List[Column],
    rows: t.List[Row],
    batch_size: int = 50
) -> t.Dict:
    """Create an experiment with columns and rows.
    
    This method creates an experiment and populates it with columns and rows in an
    optimized way using concurrent requests.
    
    Args:
        project_id: Project ID
        name: Experiment name
        description: Experiment description
        columns: List of column definitions
        rows: List of row data
        batch_size: Number of operations to perform concurrently
        
    Returns:
        The created experiment
    """
    return await self._create_with_data(
        "experiment", project_id, name, description, columns, rows, batch_size
    )

In [ ]:
#| export
#---- Utility Methods ----
@patch
def create_column(
    self: RagasRelay, 
    name: str, 
    type: str, 
    settings: t.Optional[t.Dict] = None, 
    col_order: t.Optional[int] = None,
    id: t.Optional[str] = None
) -> Column:
    """Create a Column object.
    
    Args:
        name: Column name
        type: Column type (use ColumnType enum)
        settings: Column settings
        col_order: Column order
        id: Custom ID (generates one if not provided)
        
    Returns:
        Column object
    """
    params = {"name": name, "type": type}
    if settings:
        params["settings"] = settings
    if col_order is not None:
        params["col_order"] = col_order
    if id:
        params["id"] = id
        
    return Column(**params)
    
@patch
def create_row(
    self: RagasRelay, 
    data: t.Dict[str, t.Any], 
    column_map: t.Dict[str, str],
    id: t.Optional[str] = None
) -> Row:
    """Create a Row object from a dictionary.
    
    Args:
        data: Dictionary mapping column names to values
        column_map: Dictionary mapping column names to column IDs
        id: Custom ID (generates one if not provided)
        
    Returns:
        Row object
    """
    cells = []
    for col_name, value in data.items():
        if col_name in column_map:
            cells.append(RowCell(
                data=value,
                column_id=column_map[col_name]
            ))
            
    params = {"data": cells}
    if id:
        params["id"] = id
        
    return Row(**params)
    
@patch
def create_column_map(self: RagasRelay, columns: t.List[Column]) -> t.Dict[str, str]:
    """Create a mapping of column names to IDs.
    
    Args:
        columns: List of column objects
        
    Returns:
        Dictionary mapping column names to IDs
    """
    return {col.name: col.id for col in columns}
    
@patch
async def convert_raw_data(
    self: RagasRelay,
    column_defs: t.List[t.Dict],
    row_data: t.List[t.Dict]
) -> t.Tuple[t.List[Column], t.List[Row]]:
    """Convert raw data to column and row objects.
    
    Args:
        column_defs: List of column definitions (dicts with name, type)
        row_data: List of dictionaries with row data
        
    Returns:
        Tuple of (columns, rows)
    """
    # Create columns
    columns = []
    for col in column_defs:
        columns.append(self.create_column(**col))
        
    # Create column map
    column_map = self.create_column_map(columns)
    
    # Create rows
    rows = []
    for data in row_data:
        rows.append(self.create_row(data, column_map))
        
    return columns, rows